In [1]:
!pip uninstall -y qiskit qiskit-terra qiskit-aer qiskit-ibmq-provider 

Found existing installation: qiskit 1.2.0
Uninstalling qiskit-1.2.0:
  Successfully uninstalled qiskit-1.2.0
Found existing installation: qiskit-aer 0.15.0
Uninstalling qiskit-aer-0.15.0:
  Successfully uninstalled qiskit-aer-0.15.0


In [2]:
pip list

Package                   Version
------------------------- --------------
annotated-types           0.7.0
anyio                     4.4.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     24.2.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.1.0
certifi                   2024.7.4
cffi                      1.17.0
charset-normalizer        3.3.2
colorama                  0.4.6
comm                      0.2.2
cryptography              43.0.0
debugpy                   1.8.5
decorator                 5.1.1
defusedxml                0.7.1
dill                      0.3.8
executing                 2.0.1
fastjsonschema            2.20.0
fqdn                      1.5.1
h11                       0.14.0
httpcore                  1.0.5
httpx                     0.27.0
ibm-cloud-sdk-core        3.20.6
ibm-platform-se

In [3]:
!pip install qiskit

  Using cached qiskit-1.2.0-cp38-abi3-win_amd64.whl.metadata (13 kB)
Using cached qiskit-1.2.0-cp38-abi3-win_amd64.whl (4.6 MB)


In [4]:
pip install qiskit-aer

  Using cached qiskit_aer-0.15.0-cp311-cp311-win_amd64.whl.metadata (8.2 kB)
Using cached qiskit_aer-0.15.0-cp311-cp311-win_amd64.whl (9.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import qiskit
print(qiskit.__version__)

1.2.0


In [6]:
pip install qiskit-ibm-runtime

In [7]:
from qiskit_ibm_runtime import QiskitRuntimeService
QiskitRuntimeService.save_account(channel="ibm_quantum", token="0536bb05a1faf7335d3d9e3132887ad9b3217ed458501bc6c2eeffc697dd93cc3fd3035952421d9ea6d6d1828b2d0dcb76bb1db6cef0131b44600b730739a9b2")

AccountAlreadyExistsError: 'Named account (default-ibm-quantum) already exists. Set overwrite=True to overwrite.'

In [8]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler

class QuantumResourceAuction:
    def __init__(self, resources, users, bids):
        self.resources = resources
        self.users = users
        self.bids = bids
        self.allocations = {}
        self.payments = {user: 0 for user in users}

    def run_auction(self):
        for r in self.resources:
            highest_bid = -float('inf')
            second_highest_bid = -float('inf')
            winner = None
            
            # Determine the highest and second-highest bids
            for u in self.users:
                bid = self.bids[u][r]
                if bid > highest_bid:
                    second_highest_bid = highest_bid
                    highest_bid = bid
                    winner = u
                elif bid > second_highest_bid:
                    second_highest_bid = bid
            
            # Allocate the resource to the highest bidder
            self.allocations[r] = winner
            # Set the payment as the second-highest bid
            self.payments[winner] += second_highest_bid

    def print_results(self):
        print("Resource Allocations:")
        for r, u in self.allocations.items():
            print(f"Resource {r} is allocated to User {u}")
        
        print("\nUser Payments:")
        for u, payment in self.payments.items():
            print(f"User {u} pays {payment}")

    def run_quantum_job(self):
        # Initialize Qiskit Runtime Service
        service = QiskitRuntimeService()

        # Dynamically select the least busy backend
        backend = service.least_busy(min_num_qubits=5, filters=lambda x: not x.configuration().simulator)

        if backend is None:
            raise QiskitBackendNotFoundError("No suitable backend found.")

        print(f"Selected Backend: {backend.name}")

        # Create a simple quantum circuit
        qc = QuantumCircuit(5, 5)
        qc.h(0)
        qc.cx(0, 1)
        qc.cx(0, 2)
        qc.cx(0, 3)
        qc.cx(0, 4)
        qc.measure(range(5), range(5))

        # Transpile the circuit for the selected backend
        transpiled_qc = transpile(qc, backend=backend)

        # Run the quantum job in a runtime session
        with Session(backend=backend) as session:
            sampler = Sampler(backend=backend, mode="runtime")
            result = sampler.run([transpiled_qc]).result()  # Wrap `transpiled_qc` in a list

        # Extract and print the results
        counts = result[0].binary_probabilities()  # Get binary probabilities
        print("Job Result:", counts)

# Test Cases

# Test 1: Simple Case
print("Test 1: Simple Case")
resources = ['Qubit1', 'Qubit2', 'Qubit3']
users = ['UserA', 'UserB', 'UserC']

bids = {
    'UserA': {'Qubit1': 10, 'Qubit2': 15, 'Qubit3': 5},
    'UserB': {'Qubit1': 20, 'Qubit2': 10, 'Qubit3': 25},
    'UserC': {'Qubit1': 30, 'Qubit2': 35, 'Qubit3': 10}
}

auction = QuantumResourceAuction(resources, users, bids)
auction.run_auction()
auction.print_results()

# Expected Output:
# Resource Allocations:
# Resource Qubit1 is allocated to UserC
# Resource Qubit2 is allocated to UserC
# Resource Qubit3 is allocated to UserB
#
# User Payments:
# UserA pays 0
# UserB pays 10
# UserC pays 20

# Test 2: Case with Tied Bids
print("\nTest 2: Case with Tied Bids")
resources = ['Qubit1', 'Qubit2', 'Qubit3']
users = ['UserA', 'UserB', 'UserC']

bids = {
    'UserA': {'Qubit1': 10, 'Qubit2': 15, 'Qubit3': 20},
    'UserB': {'Qubit1': 10, 'Qubit2': 15, 'Qubit3': 20},
    'UserC': {'Qubit1': 30, 'Qubit2': 35, 'Qubit3': 10}
}

auction = QuantumResourceAuction(resources, users, bids)
auction.run_auction()
auction.print_results()

# Expected Output:
# Resource Allocations:
# Resource Qubit1 is allocated to UserC
# Resource Qubit2 is allocated to UserC
# Resource Qubit3 is allocated to UserA
#
# User Payments:
# UserA pays 20
# UserB pays 0
# UserC pays 10

# Test 3: Case with No Bids for a Resource
print("\nTest 3: Case with No Bids for a Resource")
resources = ['Qubit1', 'Qubit2', 'Qubit3']
users = ['UserA', 'UserB', 'UserC']

bids = {
    'UserA': {'Qubit1': 0, 'Qubit2': 0, 'Qubit3': 0},
    'UserB': {'Qubit1': 20, 'Qubit2': 10, 'Qubit3': 0},
    'UserC': {'Qubit1': 0, 'Qubit2': 0, 'Qubit3': 10}
}

auction = QuantumResourceAuction(resources, users, bids)
auction.run_auction()
auction.print_results()

# Expected Output:
# Resource Allocations:
# Resource Qubit1 is allocated to UserB
# Resource Qubit2 is allocated to UserB
# Resource Qubit3 is allocated to UserC
#
# User Payments:
# UserA pays 0
# UserB pays 0
# UserC pays 0

# If you want to run the quantum job, uncomment the following line.
auction.run_quantum_job()

Test 1: Simple Case
Resource Allocations:
Resource Qubit1 is allocated to User UserC
Resource Qubit2 is allocated to User UserC
Resource Qubit3 is allocated to User UserB

User Payments:
User UserA pays 0
User UserB pays 10
User UserC pays 35

Test 2: Case with Tied Bids
Resource Allocations:
Resource Qubit1 is allocated to User UserC
Resource Qubit2 is allocated to User UserC
Resource Qubit3 is allocated to User UserA

User Payments:
User UserA pays 20
User UserB pays 0
User UserC pays 25

Test 3: Case with No Bids for a Resource
Resource Allocations:
Resource Qubit1 is allocated to User UserB
Resource Qubit2 is allocated to User UserB
Resource Qubit3 is allocated to User UserC

User Payments:
User UserA pays 0
User UserB pays 0
User UserC pays 0
Selected Backend: ibm_kyoto


C:\Users\Bodhdipta\AppData\Local\Temp\ipykernel_17428\1591926089.py:68: DeprecationWarning: The 'backend' keyword arguments are deprecated as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. Please use the 'mode' parameter instead.
  sampler = Sampler(backend=backend, mode="runtime")
C:\Users\Bodhdipta\AppData\Local\Temp\ipykernel_17428\1591926089.py:156: DeprecationWarning: The backend name as execution mode input has been deprecated. as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. A backend object should be provided instead. Get the backend directly from the service using `QiskitRuntimeService().backend('ibm_backend')`
  auction.run_quantum_job()
C:\Users\Bodhdipta\AppData\Local\Temp\ipykernel_17428\1591926089.py:68: DeprecationWarning: Passing a backend as a string is deprecated as of qiskit-ibm-runtime 0.26.0 and will be removed no sooner than 3 months after the release date. Use the

QiskitBackendNotFoundError: 'No backend matches the criteria.'